In [76]:
class Graph:
    pass

class User:
    pass

# Touch Typing

Alec Reiter | PyAtl  January 2019

@just_anr http://justanr.github.io

## Let's not start a flame war...

- Static vs Dynamic
- Both have benefits

## A brief history of Type Annotations

* Added in 3.0 (PEP 3107)
* Codified in 3.5 (PEP 484)
* Extended in 3.6 (PEP 526)

## Static Typing for Dynamic Python

* Gradual
* Not checked by interpreter
* Preserved at runtime

## Benefits

* Bugs
* Refactoring
* Documentation
* Metaprogramming

## Drawbacks

- Not the most expressive
- Verbose
- Not Completely 1st Class Yet

## What's it look like?

## Function Parameter Types

In [48]:
def wrap_in_parens(a: str, default: str = '') -> str:
    return '(' + a.strip() + ')'

## lambda

- `lambda x: x`
- Can't type hint lambdas in line

## Variable Types

In [83]:
a: int
a: int = 1
a = 1

## Attribute Types

In [53]:
class Point:
    x: int
    y: int

## Collections

In [54]:
def plot_points(points: list) -> Graph:
    ...

## Typing Module

In [74]:
import typing as t

* Added in 3.5
* More complex types
* Generics, NewType, Union, and more

## Collections

In [55]:
def plot_points(points: t.List[Point]) -> Graph:
    ...

## Generics

In [59]:
T = t.TypeVar('T')

class SearchResult(t.Generic[T]):
    results: t.List[T]
    has_more: bool

## Class Attributes

In [ ]:
class Foo:
    x: t.ClassVar[int] = 1

## Callable Types

```python
t.Callable[[int, int], int]

lambda x: x  # type: t.Callable[[str], str]
```

## Aliases

In [ ]:
Callable[[AbstractConverter, Tuple[type], Dict[str, Any]], FieldABC]

In [ ]:
FieldFactory = Callable[[AbstractConverter, Tuple[type], Dict[str, Any]], FieldABC]

## Forward References

In [61]:
class LinkedList(t.Generic[T]):
    value: T
    next: t.Optional['LinkedList[T]']

## Unions

In [75]:
t.Union[int, str]
t.Optional[int]

typing.Union[int, NoneType]

## New Type

In [56]:
UserId = t.NewType('UserId', int)

In [57]:
def find_user(id: UserId) -> User:
    ...

```python
find_user(1)
```

`Argument 1 to "find_user" has incompatible type "int"; expected "UserId"`

In [58]:
find_user(UserId(1))

## Casting

In [73]:
l: object = [1]
x = t.cast(t.List[int], l)

## What about Python 2?

- Just upgrade

### Stub Files

- "Header" files
- Python 2 and 3 compat
- Add types to untyped modules
- Typeshed repo

In [ ]:
# unittest.pyi
class TestSuite(Testable):
    def __init__(self, tests: Iterable[Testable] = None) -> None: ...
    def addTest(self, test: Testable) -> None: ...
    def addTests(self, tests: Iterable[Testable]) -> None: ...
    def run(self, result: TestResult) -> None: ...
    def debug(self) -> None: ...
    def countTestCases(self) -> int: ...

## AntiPatterns

## Not Specific Enough

In [63]:
class SearchResults:
    results: t.List[object]

- typing.Any, type, object
- _Sometimes_ the right types
- type -> typing.Type[T]
- object -> T

## Too Specific

In [66]:
def add_two(x: int, y: int) -> int:
    return x + y

In [67]:
from typing import TypeVar
from decimal import Decimal

AnyNum = TypeVar('AnyNum', int, float, complex, Decimal)

def add_two(x: AnyNum, y: AnyNum) -> AnyNum:
    return x + y

```
add_two(1, 1)
add_two(1.1, 1.1)
add_two(1j, 1j)
add_two(Decimal(1), Decimal(1))
```

In [68]:
add_two(1, Decimal(1))

Decimal('2')

`error: Value of type variable "AnyNum" of "add_two" cannot be "object"`

## Typing Everything

- Inference
- Biting off too much at once
- _Sometimes_ untyped non-public is more helpful

## Cool, but it's not enforced at runtime...

## Enter mypy

* Offline type checker
* Think "flake8" for types
* https://mypy.readthedocs.io

```python
wrap_in_parens(1)
```

`Argument 1 to "wrap_in_parens" has incompatible type "int"; expected "str"`

## Mypy Pragmas

- `# type: int`
- `# type: ignore`
- `reveal_type`

### Configuring

- tons of settings
- cli arguments
- mypy.ini

### Example

```
[mypy]

ignore_missing_imports = False
show_column_numbers = True
show_error_context = False
follow_imports = normal
cache_dir = /dev/null
disallow_untyped_calls = False
warn_return_any = True
strict_optional = True
warn_no_return = True
warn_redundant_casts = True
warn_unused_ignores = False
disallow_untyped_defs = False
check_untyped_defs = True
```

```
[mypy-marshmallow.*]
ignore_missing_imports = True
```

## As part of your build

```
# tox.ini
[testenv:types]
skip_install = false
deps = -r{toxinidir}/requirements/requirements-types.txt
commands = mypy ./src/marshmallow_annotations
```

```
# .travis.yml

matrix:
  include:
    - python: 3.6
      env: TOXENV=types
```

# Using Types Programatically

## Your new best friend

In [70]:
t.get_type_hints(Point)

{'x': int, 'y': int}

## Lots of things we can do

- Serialization Formats
- Code Generation
- Dependency Injection
- Documentation

### Injector

- IoC Container
- Type hints

In [ ]:
from injector import inject
from myapp.database import Connection

class UserService:
    @inject
    def __init__(self, conn: Connection) -> None:
        self.conn = conn

In [ ]:
from injector import Module, provider
from myapp.database import Connection
from myapp.users import UserService

class AppModule(Module):
    @provider
    def get_connection() -> Connection:
        return Connection(...)
    
    def configure(self, binder) -> None:
        binder.bind(UserService)

In [ ]:
from injector import Injector
from myapp.ioc import AppModule

ioc = Injector([AppModule()])
ioc.get(UserService)

## Code Generation

- Tedious `__init__`
- Not interesting
- Just generate it

In [29]:
from dataclasses import dataclass

@dataclass
class Foo:
    a: int
    b: float
    c: str = 'yep'
        
Foo(1, 2.0)

Foo(a=1, b=2.0, c='yep')

## More complicated

- Post Init
- non-static defaults
- non-attribute init
- ordering

### Serialization Formats

- Issue: Entities and JSON
- Write everything twice?
- Nah

In [ ]:
@dataclass
class Artist:
    id: int
    name: str
    albums: t.List['Album'] = field(default_factory=list)

In [ ]:
class ArtistScheme(Schema):
    id = fields.Integer(required=True, allow_none=False)
    name = fields.String(required=True, allow_none=False)
    albums = fields.Nested('AlbumScheme', many=True, required=True, allow_none=False)

In [ ]:
from marshmallow_annotations import AnnotationSchema
from .music import Artist

class ArtistScheme(AnnotationSchema):
    class Meta:
        target = Artist
        register_as_scheme = True

### Documentation

`pip install --user sphinx-autodoc-annotation`

In [80]:
# conf.py

extensions = ["sphinx_autodoc_annotation", ...]

In [ ]:
def register_field_for_type(self, target: type, field: FieldABC) -> None:
    ...

```
register_field_for_type(target: type, field: marshmallow.base.FieldABC) → None
```

## Don't fear the types

- Useful
- Somewhat intrusive
- Still dynamic

# Questions?